In [1]:
# !pip install ipywidgets  # for vscode
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import json

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
from transformers import AutoTokenizer, AutoModel

from sklearn.utils.class_weight import compute_class_weight

In [2]:
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModel.from_pretrained("monologg/kobert")

In [65]:
class sentencEmojiDataset(Dataset):
    def __init__(self, directory, tokenizer):
        
        data = pd.read_csv(directory, encoding='UTF-8')
        
        self.tokenizer = tokenizer
        self.sentences = list(data.iloc[:,0])
       
        emojis = list(data.iloc[:,1])
        emojis_unique = list(set(emojis))
        
        self.labels = [emojis_unique.index(i) for i in emojis]
 
        self.labels_dict = {'key': range(len(emojis_unique)), 'value': emojis_unique}
        
    def __getitem__(self, i): #collate 이전 미리 tokenize를 시켜주자
        tokenized = self.tokenizer(str(self.sentences[i]), return_tensors='pt')
        
        #아래 세 개는 tokenizer가 기본적으로 반환하는 정보. BERT의 input이기도 함
        input_ids = tokenized['input_ids']
        token_type_ids = tokenized['token_type_ids']
        attention_mask = tokenized['attention_mask']
        
        print(str(self.sentences[i]) +' : ')
        print(tokenized)
        
        return {'input_ids': input_ids, 'token_type_ids': token_type_ids, 
                'attention_mask': attention_mask, 'label': self.labels[i]}
         
    def __len__(self): #data loader가 필요로 하여 필수적으로 있어야 하는 함수
        return len(self.sentences)

In [66]:
class collate_fn:
    def __init__(self, labels_dict):
        self.num_labels = len(labels_dict)
        
    def __call__(self, batch): #batch는 dataset.getitem의 return 값의 List. eg. [{}, {}. ...]
        #batch내 최대 문장 길이(토큰 개수)를 먼저 구해서 padding할 수 있도록 하기
        batchlen = [sample['input_ids'].size(1) for sample in batch] #tensor값을 반환하기 때문에 1번째 차원의 길이를 구함
        maxlen = max(batchlen)
        input_ids = []
        token_type_ids = []
        attention_mask = []
        #padding: [5, 6] [0, 0,  ...]을 concatenate 하는 방식으로 패딩
        for sample in batch:
            pad_len = maxlen - sample['input_ids'].size(1)
            pad = torch.zeros((1, pad_len), dtype=torch.int)            
            input_ids.append(torch.cat([sample['input_ids'], pad], dim=1))
            token_type_ids.append(torch.cat([sample['token_type_ids'], pad], dim=1))
            attention_mask.append(torch.cat([sample['attention_mask'], pad], dim=1))
        #batch 구성
        input_ids = torch.cat(input_ids, dim=0)
        token_type_ids = torch.cat(token_type_ids, dim=0)
        attention_mask = torch.cat(attention_mask, dim=0)
        
        #one-hot encoding
        #batch 내 라벨을 tensor로 변환
        tensor_label = torch.tensor([sample['label'] for sample in batch])
        
        return input_ids, token_type_ids, attention_mask, tensor_label

In [67]:
df = pd.read_csv('data/twitter_clean.csv', encoding="UTF-8")
print(len(df['y'].value_counts()))
df['y'].value_counts(sort = True).head(10)

30


N    235
😂     80
❤     53
😭     49
😍     32
💕     28
💜     26
🔥     25
👍     24
🥺     24
Name: y, dtype: int64

In [68]:
df['split'] = np.random.randn(df.shape[0], 1)
msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [69]:
train = sentencEmojiDataset('data/train.csv', tokenizer)
test = sentencEmojiDataset('data/test.csv', tokenizer)

train_collate_fn = collate_fn(train.labels_dict)
test_collate_fn = collate_fn(test.labels_dict)

train_collate_fn

In [70]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-3

In [71]:
train_dataloader = DataLoader(train, batch_size=batch_size, collate_fn=train_collate_fn, shuffle = True, drop_last = True)
test_dataloader = DataLoader(test, batch_size=batch_size, collate_fn=test_collate_fn, shuffle = False, drop_last = False)
train_dataloader

In [72]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        # do not train bert parameters
        for p in self.bert.parameters():
            p.requires_grad = False
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, input_ids, token_type_ids, attention_mask):
        #eval: drop out 중지, batch norm 고정과 같이 evaluation으로 모델 변경
        self.bert.eval()
        #gradient 계산을 중지
        with torch.no_grad():
            x = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
#         x = self.dropout(pooler)
        return self.classifier(x)

In [73]:
label = list(set(list(df.iloc[:,1])))

In [74]:
model = BERTClassifier(model,  dr_rate=0.5, num_classes = len(label))

In [75]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [76]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# optimizer = AdamW(model.parameters(), lr=learning_rate)

In [77]:
np.unique(train.labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [78]:
train.labels[:20]

[19,
 15,
 19,
 19,
 19,
 19,
 19,
 15,
 26,
 18,
 15,
 19,
 21,
 19,
 19,
 19,
 15,
 19,
 19,
 21]

In [79]:
#Class Imbalance 문제 해결을 위한 weighted cross entropy 
class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train.labels), y = train.labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights) #([1.0000, 1.0000, 4.0000, 1.0000, 0.5714])

tensor([1.1807, 0.7010, 1.8694, 2.4926, 1.7256, 1.1217, 1.3196, 1.7256, 1.2463,
        2.2433, 2.2433, 1.8694, 3.2048, 1.0197, 1.1217, 0.4314, 2.0394, 2.4926,
        1.3196, 0.1297, 1.3196, 0.6231, 1.1807, 1.1807, 1.3196, 1.7256, 1.3196,
        1.6024, 1.8694, 1.7256])


In [80]:
train.labels_dict

{'key': range(0, 30),
 'value': ['🙏',
  '😭',
  '💯',
  '🐹',
  '😁',
  '💖',
  '😆',
  '💗',
  '👍',
  '✨',
  '👉',
  '😑',
  '🤤',
  '💕',
  '😍',
  '😂',
  '😎',
  '✌',
  '🥰',
  'N',
  '😊',
  '❤',
  '❣',
  '💜',
  '😘',
  '🤭',
  '🥺',
  '🔥',
  '💙',
  '💚']}

In [81]:
loss_fn = nn.CrossEntropyLoss(weight = class_weights, reduction = 'mean') 

In [82]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [83]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [84]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [85]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    loss_sum = 0
    model.train()
    for batch_id, (input_ids, token_type_ids, attention_mask, tensor_label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        out = model(input_ids, token_type_ids, attention_mask)
        loss = loss_fn(out, tensor_label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
#         scheduler.step()  # Update learning rate schedule
        batch_acc = calc_accuracy(out, tensor_label)
        train_acc += batch_acc
        loss_sum += loss.data.cpu().numpy()
        #f batch_id % log_interval == 0:
        print("epoch {} batch id {}/{} loss {} train acc {}".format(e+1, batch_id+1, len(train_dataloader), loss.data.cpu().numpy(), batch_acc))
    print("epoch {} train acc {} loss mean {}".format(e+1, train_acc / (batch_id+1), loss_sum / len(train_dataloader)))
    model.eval()
    with torch.no_grad():
        for batch_id, (input_ids, token_type_ids, attention_mask, tensor_label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
            out = model(input_ids, token_type_ids, attention_mask)
            test_acc += calc_accuracy(out, tensor_label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

월이 너무 기다려져요 : 
{'input_ids': tensor([[2, 0, 0, 0, 3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
다음에두 엘다님이 맘찍해둘 수 있는 예쁜곳을 만들어보겠어욥 배세진생일가뵤자고세진이의눈부시고도아름다운제막배세진 생일 카페에 협력한 그림이에요생일축하해 : 
{'input_ids': tensor([[   2,    0,    0,    0, 6629, 7142,    0,    0,    0,    0,    0,    0,
            0,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
해적 도깨비 깃발 출장십오야열정 해적선 단원들과 : 
{'input_ids': tensor([[2, 0, 0, 0, 0, 0, 0, 3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
신고해도 별 소용도 없나봐신발은 다들 어케 보관 하시지 : 
{'input_ids': tensor([[   2,    0, 6356,    0,    0,    0,    0,    0,    0,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
민서에게 나는 당신의 차선으로 끌려가고 있다고 말하고 싶어요 사랑해요 : 
{'input_ids': tensor([[   2,    0, 

AttributeError: 'Tensor' object has no attribute 'pooler_output'